### Hyperparams

In [24]:
import torch

batch_size = 32
block_size = 8
max_iters = 3_000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200
train_split = 0.9
device = 'cuda'

### Data

In [6]:
with open ('/data/tinyshakespeare.txt', 'r', encoding='utf-8') as file:
    text = file.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s] # encoder: map string to char ids
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: map integer ids to a string

data = torch.tensor(encode(text), dtype=torch.long)

# Splitting the dataset into training and validation sets
n = int(train_split * len(data)) # 90% training, 10% validation
train_data, validation_data = data[:n], data[n:]

def get_batch(split):
  # batch of data of inputs x and targets y
  data = train_data if split == 'train' else validation_data
  # array(batch_size) of random offsets within data
  ix = torch.randint(len(data) - block_size, (batch_size,))

  # Context and target sequences
  # ? Why a single sequence is not used?
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  x, y = x.to(device), y.to(device)

  return x, y

### Loss

In [8]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()

  for split in ['train', 'validation']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()

    out[split] = losses.mean()

  model.train()
  return out

### Model

In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()

    # Video says: each token directly reads off the logits for the next token
    # from the lookup table.
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # idx and targets are (B, T) tensors of integers, Batch, Time
    # Token embedding table will return shape of (Batch, Time, Channel)
    # Batch - number of sequences
    # Time - number of tokens in each sequence
    # Channel - Embeddings vector for a token?
    logits = self.token_embedding_table(idx)

    if targets is None:
      return logits, None

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)

    # Cross entropy expects input to be in a 2D tensor of shape (Batch*Time, Channel)
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is a (Batch, Time) tensor of integers, representing current context
    for _ in range(max_new_tokens):
      # compute the predictions
      logits, loss = self(idx) # (B, T, C)

      # -1 makes very little sense for a bigram model,
      # as we're essentially throwing away everything except the very last token in a batch
      # to make our prediction.
      # This is done in such a way only to allow for easier transition to an N-gram model later.
      logits = logits[:, -1, :] # (B, C)

      # Probabilities from logits
      # Softmax ~ [1, 2, 3, 4]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

      # append to the currently running context
      idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
    return idx

model = BigramLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

### Train

In [30]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f'Iter {iter}, Train loss: {losses["train"]}, Validation loss: {losses["validation"]}')

  # Sample batch of data
  xb, yb = get_batch('train')

  # Eval
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

Iter 0, Train loss: 4.762716770172119, Validation loss: 4.763288974761963
Iter 300, Train loss: 2.8414525985717773, Validation loss: 2.8634636402130127
Iter 600, Train loss: 2.5515248775482178, Validation loss: 2.5881149768829346
Iter 900, Train loss: 2.4970433712005615, Validation loss: 2.5305380821228027
Iter 1200, Train loss: 2.490762233734131, Validation loss: 2.5049004554748535
Iter 1500, Train loss: 2.4716250896453857, Validation loss: 2.5027894973754883
Iter 1800, Train loss: 2.45572566986084, Validation loss: 2.4971234798431396
Iter 2100, Train loss: 2.4724764823913574, Validation loss: 2.4942057132720947
Iter 2400, Train loss: 2.463688611984253, Validation loss: 2.4955973625183105
Iter 2700, Train loss: 2.4643714427948, Validation loss: 2.489426374435425
Iter 3000, Train loss: 2.4624266624450684, Validation loss: 2.493394374847412
Iter 3300, Train loss: 2.460221290588379, Validation loss: 2.493234634399414
Iter 3600, Train loss: 2.454041004180908, Validation loss: 2.4851751327

### Generate

In [32]:
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
batch = model.generate(ctx, max_new_tokens=500)
logits = batch[0].tolist()

print(decode(logits))


ONS:
GSored ve y ube I thellurs:
DWAncas syshapy d tr wn, thirdsto;
PRI d:
ARard.
Pr have S yorves,
HET:
I's, mepre be bode ave kes,
Whingerer t, ardset
Thin plun, he, ars len; mecowirund ENThayoncerkie buse myo me hot. I'sperifive l I:
We!
I heree yos

Theme!'l boanf end
Weit I tesueoomollcthe spt t:
I myscer oucond andrinu, eromerint. wh byo: H:
UCI aned n amerd icknen. bonem, ws, h tonghot nd, w, fieacou t meer, pele mabry f ch by amach's;
Andfube s oumist quo me omorn-s
O:
Cllatrnco F owind 
